In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
from peft import PeftModel
import torch

#have note book ignore warnings
import warnings
warnings.filterwarnings('ignore')


/home/bphilipose/miniconda3/envs/LegalEaseWSL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()

cuda


In [3]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map={"": 0}, trust_remote_code=False, revision="main")

 

CUDA extension not installed.
CUDA extension not installed.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: {'model.layers.17.self_attn.o_proj.bias', 'model.layers.3.mlp.up_proj.bias', 'model.layers.14.mlp.gate_proj.bias', 'model.layers.30.self_attn.k_proj.bias', 'model.layers.11.self_attn.q_proj.bias', 'model.layers.13.mlp.down_proj.bias', 'model.layers.14.self_attn.k_proj.bias', 'model.layers.26.mlp.down_proj.bias', 'model.layers.4.mlp.up_proj.bias', 'model.layers.9.mlp.down_proj.bias', 'model.layers.18.self_attn.v_proj.bias', 'model.layers.31.self_attn.v_proj.bias', 'model.layers.18.mlp.gate_proj.bias', 'model.layers.2.mlp.gate_proj.bias', 'model.layers.21.self_attn.q_proj.bias', 'model.layers.4.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.5.self_attn.q_proj.bias', 

In [4]:


# Apply tokenization
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


In [5]:
comment = "I want to start a foodbank in Washington, how do I?"

### Prompt with instructions

In [6]:
instructions_string = f"""You are an expert business consultant designed to help users determine the most suitable business structure for their needs. You will guide the user toward choosing one of the following options:
- **LLC (Limited Liability Company)**
- **Non-Profit Organization**
- **S-Corporation**

Do not recommend General Partnerships, Sole Proprietorships, or C-Corporations.

### **Your Approach**
1. **Ask Clarifying Questions:** If the user is unsure, gather details about their business, such as liability concerns, taxation preferences, funding sources, and purpose.
2. **Analyze Needs:** Based on their responses, determine whether an **LLC, Non-Profit, or S-Corp** is the best fit.
3. **Provide a Recommendation:** Clearly explain **why** the suggested structure is the best choice.
4. **Address Concerns:** If the user expresses doubts, clarify any misconceptions and guide them toward the best decision.

### **Response Format**
- **Direct Answer:** If the user describes their business clearly, provide a structured recommendation.
- **Guided Questions:** If the user is unsure, ask simple questions to help them decide.
- **Comparison (if needed):** If two structures could work, briefly compare them.

### **Example Scenarios**
#### **User Input 1:**  
*"I’m starting a consulting business with a partner. We want liability protection and pass-through taxation."*  
**LLM Response:**  
*"An LLC would be ideal for you. It offers **liability protection**, meaning your personal assets are safe, and it allows **pass-through taxation**, so profits aren’t taxed twice. You’ll also have flexibility in management and ownership. Would you like help with the next steps?"*

#### **User Input 2:**  
*"I want to create an organization to help underprivileged kids get scholarships."*  
**LLM Response:**  
*"A Non-Profit is the best choice for your mission. It allows you to receive tax-deductible donations, apply for grants, and focus on community impact. To qualify, your organization must serve the public interest. Do you need help with 501(c)(3) registration?"*

#### **User Input 3:**  
*"I run a small business and want liability protection, but I also want to save on self-employment taxes."*  
**LLM Response:**  
*"An S-Corporation might be your best option. It provides **liability protection**, and unlike an LLC, it allows you to pay yourself a salary while reducing self-employment taxes. Would you like a quick breakdown of how S-Corp taxation works?"*

**Now, respond to the following user inquiry:**
"""
   

In [7]:
     
#prompt=f'''[INST] {comment} [/INST]'''

prompt_template = lambda in_comment: f'''[INST] {instructions_string} \n{in_comment} \n[/INST]'''
prompt = prompt_template(comment)

In [8]:
model.eval()


# Tokenize the input text
inputs = tokenizer(prompt, return_tensors="pt")

#generate output
outputs = model.generate(input_ids=inputs["input_ids"].to('cuda'), max_new_tokens=500)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [9]:
text_output = tokenizer.batch_decode(outputs)[0]

#break on [/INST]
text_output = text_output.split("[/INST]")[1]
print(text_output)


 **LLM Response:**

Starting a food bank in Washington involves several steps to establish a non-profit organization. Here's a simplified guide:

1. **Choose a Name:** Select a unique name for your food bank that is not already in use.
2. **Create a Board of Directors:** Assemble a board of directors to oversee the organization's operations.
3. **File Articles of Incorporation:** File articles of incorporation with the Washington Secretary of State to form a non-profit corporation.
4. **Apply for Tax-Exempt Status:** Apply for tax-exempt status from the Internal Revenue Service (IRS) as a 501(c)(3) organization.
5. **Register with the Washington State Department of Revenue:** Register with the Washington State Department of Revenue to collect and report sales tax.
6. **Obtain Necessary Permits and Licenses:** Obtain any necessary permits and licenses, such as a food service permit, from local and state agencies.
7. **Set Up a Bank Account:** Set up a bank account for your non-profit or

## Prepping model for training

In [10]:
model.train()

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

####Set up LoRA

In [11]:
#LoRA config
config = LoraConfig(r=8, lora_alpha=32, target_modules=["q_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()



trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


In [12]:
custom_data = load_dataset("json", data_files="../../datasets/qa_pairs.json")

split_dataset = custom_data["train"].train_test_split(test_size=0.1)

def format_prompt(example):
    instruction = example["instruction"]
    response = example["response"]
    prompt = f"[INST] {instruction} [/INST]"
    
    full_text = f"{prompt}\n{response}"
    return {"example": full_text}
data = split_dataset.map(format_prompt).remove_columns(["instruction", "response"])
#drop instruction and response columns



Map: 100%|██████████| 62/62 [00:00<00:00, 15715.65 examples/s]


In [13]:
print(data)
# Print first 5 rows of the training dataset
for i in range(5):
    print(data["train"][i])  # Prints each row
    print("="*50)  # Separator for readability

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 558
    })
    test: Dataset({
        features: ['example'],
        num_rows: 62
    })
})
{'example': '[INST] How does an LLC differ from a corporation in Washington? [/INST]\nLLCs offer more flexibility in management and typically simpler taxation3.'}
{'example': '[INST] Can a PLLC have non-professional members? [/INST]\nGenerally no, all members must be licensed in the relevant profession.'}
{'example': '[INST] Can an LLC have different classes of membership? [/INST]\nYes, LLCs can  create different classes of membership with varying rights and privileges.'}
{'example': '[INST] Can a Washington LLC operate in other states? [/INST]\nYes, but you must register as a foreign LLC in each state where you conduct  business.'}
{'example': '[INST] What do I need to do if I want to obtain a federal tax number for my LLC or  corporation?\u200b [/INST]\nYou’ll need to obtain a federal ID number (EIN) from the 

In [14]:
#create tokenize functoin
def tokenize_function(examples):
    #extract text
    text = examples["example"]
    
    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(text, return_tensors="np", truncation=True, max_length=512)
    
    
    return tokenized_inputs

#tokenize training and validation data
tokenized_data = data.map(tokenize_function, batched=True)

Map: 100%|██████████| 62/62 [00:00<00:00, 16734.03 examples/s]


In [15]:
#setting pad token
tokenizer.pad_token = tokenizer.eos_token

#data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [16]:
#hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

#define training args
training_args = transformers.TrainingArguments(
    output_dir="./output",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit"
    )
    

In [17]:
#configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
    )

#train model
model.config.use_cache = False
trainer.train()

# renable warnings
model.config.use_cache = True

Epoch,Training Loss,Validation Loss
1,2.824300,1.646378
2,1.417700,1.384386
3,1.207500,1.351078
4,1.080700,1.340305
5,0.975600,1.369766
6,0.880000,1.417781
7,0.801600,1.497388
8,0.734400,1.583008
9,0.683100,1.668267
10,0.648600,1.712029


In [18]:
# Save Model & Tokenizer
model.save_pretrained("./business_llm")
tokenizer.save_pretrained("./business_llm")

('./business_llm/tokenizer_config.json',
 './business_llm/special_tokens_map.json',
 './business_llm/tokenizer.model',
 './business_llm/added_tokens.json',
 './business_llm/tokenizer.json')

In [19]:
#stop the model and remove it from the GPU
model.cpu()
model = None
torch.cuda.empty_cache()

In [20]:

# Load Base Model
base_model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")

# Load Fine-Tuned Adapter
model = PeftModel.from_pretrained(base_model, "./business_llm")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("./business_llm", use_fast=True)

# Generate Business Type Recommendation
def generate_response(user_input):
    prompt = f"[INST] {instructions_string}\n{user_input} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    output = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test Your Model
comment = "I want to start a foodbank in Washington, how do I?"
#user_query = "I’m planning to start a small bakery where I sell homemade pastries. I want to protect my personal assets in case something goes wrong, but I also want to keep my taxes simple. I don’t expect to have any employees right away, but I might hire one or two in the future. What business structure would be best for me?"
response = generate_response(comment)
#break on [/INST]
text_output = response.split("[/INST]")[1]
print(text_output)



Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: {'model.layers.17.self_attn.o_proj.bias', 'model.layers.3.mlp.up_proj.bias', 'model.layers.14.mlp.gate_proj.bias', 'model.layers.30.self_attn.k_proj.bias', 'model.layers.11.self_attn.q_proj.bias', 'model.layers.13.mlp.down_proj.bias', 'model.layers.14.self_attn.k_proj.bias', 'model.layers.26.mlp.down_proj.bias', 'model.layers.4.mlp.up_proj.bias', 'model.layers.9.mlp.down_proj.bias', 'model.layers.18.self_attn.v_proj.bias', 'model.layers.31.self_attn.v_proj.bias', 'model.layers.18.mlp.gate_proj.bias', 'model.layers.2.mlp.gate_proj.bias', 'model.layers.21.self_attn.q_proj.bias', 'model.layers.4.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.5.self_attn.q_proj.bias', 'model.layers.19.mlp.down_proj.bias', 'model.layers.27.mlp.gate_proj.bias', 'model.layers.20.self_attn.k_proj.bias', 'model.layers.28.mlp.down_proj.bias', 'model.lay



To start a non-profit in Washington, you need to file a Non-Profit Certificate of Incorporation, create bylaws, and apply for 501(c)(3) status. This will allow you to receive tax-deductible donations and apply for grants. You'll also need to hold regular meetings and maintain records. If you plan to hire employees, you may need to register for an EIN and pay payroll taxes.

If you have any questions or need assistance, feel free to ask.

As a non-profit, you will be able to receive tax-deductible donations and apply for grants, but you will need to maintain records and hold
